In [1]:
import pickle
import warnings
import torch
import os
torch.set_default_dtype(torch.float64)

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_path = r'C:\Users\mtcd\Documents\Codes\dc3-wss\dc3\datasets\dc_wss\dc_wss_dataset_dc_5'

with open(file_path, 'rb') as f:
    data = pickle.load(f)
    
    
dir_path = r'C:\Users\mtcd\Documents\Codes\dc3-wss\dc3\models'
file_name = 'model_2025-02-10_10-22-47_dcwss_epochs_3.pt'

file_path = os.path.join(dir_path, file_name)

In [2]:
import torch.nn as nn
from functools import reduce
import operator

class NNSolver(nn.Module):
    def __init__(self, data, args):
        super().__init__()
        self._data = data
        self._args = args
        layer_sizes = [data.xdim, self._args['hiddenSize'], self._args['hiddenSize']]
        layers = reduce(operator.add,
            [[nn.Linear(a,b), nn.BatchNorm1d(b), nn.ReLU(), nn.Dropout(p=0.2)]
                for a,b in zip(layer_sizes[0:-1], layer_sizes[1:])])
        
        output_dim = data.ydim - data.nknowns        

        if self._args['useCompl']:
            layers += [nn.Linear(layer_sizes[-1], output_dim - data.neq)]            
        else:
            layers += [nn.Linear(layer_sizes[-1], output_dim)] 
            
        for layer in layers:
            if type(layer) is nn.Linear:
                nn.init.kaiming_normal_(layer.weight)

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        out = self.net(x)

        if self._args['useCompl']:
            result = self._data.complete_partial(x, out)
            return result
        else:
            result = self._data.process_output(x, out)

            return result

In [3]:
args = {'probType': 'dc_wss', 'hiddenSize': 200, 'useCompl': False}
newModel = NNSolver(data, args)
newModel.load_state_dict(torch.load(file_path))
newModel.eval()

NNSolver(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [12]:
input_data = torch.tensor([5,7,12,15,19,1,1,3,1,0.5]).unsqueeze(0)  # Adiciona uma dimensão extra
output = newModel(input_data)


In [13]:
print(output)

tensor([[24.0000,  0.0000,  0.0000, 24.0000,  0.0000,  5.6789,  6.9879,  7.8766,
          7.9836,  2.0051]], grad_fn=<CatBackward0>)


In [16]:
t_min = 0
t_max = 23

t = torch.round(int(t_min))

print(t)

TypeError: round(): argument 'input' (position 1) must be Tensor, not int